In [1]:
# Block 1: Setup + Load + sanity checks
import os
import numpy as np
import pandas as pd

from pathlib import Path

DATA_PATH = Path("/Users/asus/Downloads/MSc Final Project Stuff/outputs_final/per_track_features.csv")
OUT_DIR = Path("/Users/asus/Downloads/MSc Final Project Stuff/outputs_addendum")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def save_df(df: pd.DataFrame, name: str) -> Path:
    """Save a dataframe to the addendum output directory."""
    out_path = OUT_DIR / f"{name}.csv"
    df.to_csv(out_path, index=False)
    return out_path

# Load
df = pd.read_csv(DATA_PATH)

print("Loaded per_track_features.csv")
print("Shape:", df.shape)
print("Columns:", len(df.columns))

# Expected identifiers
expected_id_cols = {"stack", "track_id", "Genotype"}
missing = expected_id_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing expected identifier columns: {missing}")

print("\nGenotype counts:")
print(df["Genotype"].value_counts(dropna=False))

print("\nUnique stacks:", df["stack"].nunique())
print("Unique tracks:", df["track_id"].nunique())

# Basic NA overview
na_summary = (df.isna().mean().sort_values(ascending=False) * 100).round(2)
na_summary_df = na_summary.reset_index()
na_summary_df.columns = ["column", "pct_missing"]
save_df(na_summary_df, "missingness_percent_by_column")

print("\nTop 15 columns by % missing:")
print(na_summary_df.head(15))

Loaded per_track_features.csv
Shape: (4263, 60)
Columns: 60

Genotype counts:
Genotype
WT     2815
MUT    1448
Name: count, dtype: int64

Unique stacks: 16
Unique tracks: 4263

Top 15 columns by % missing:
                     column  pct_missing
0   directional_consistency        67.00
1      temporal_persistence        67.00
2                   alpha_x        58.64
3                     P_est        58.64
4                     alpha        58.64
5                   P_est_y        58.64
6                   D_est_y        58.64
7                       K_y        58.64
8                   alpha_y        58.64
9                   P_est_x        58.64
10                    D_est        58.64
11                        K        58.64
12                 speed_cv        58.64
13         area_trend_slope        58.64
14  early_eccentricity_mean        58.64


In [2]:
# Block 2: Define testable metric columns

ID_COLS = ["stack", "track_id", "Genotype"]

# Choose numeric columns only (robust to accidental strings)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# "Testable metrics" are numeric columns that aren't identifiers
metric_cols = [c for c in numeric_cols if c not in ID_COLS]

if not metric_cols:
    raise ValueError("No numeric metric columns found. Check CSV content/types.")

print("Numeric columns:", len(numeric_cols))
print("Metric columns (numeric, excluding IDs):", len(metric_cols))

# Creating a metadata table of columns to be used
metric_manifest = pd.DataFrame({
    "metric": metric_cols,
    "dtype": [str(df[m].dtype) for m in metric_cols],
    "n_missing": [int(df[m].isna().sum()) for m in metric_cols],
    "pct_missing": [(df[m].isna().mean() * 100) for m in metric_cols],
})
metric_manifest["pct_missing"] = metric_manifest["pct_missing"].round(2)

save_df(metric_manifest.sort_values("pct_missing", ascending=False), "metric_manifest")
print("Saved metric manifest.")

Numeric columns: 58
Metric columns (numeric, excluding IDs): 57
Saved metric manifest.


In [3]:
# Block 3: Distribution diagnostics (raw)

def compute_distribution_diagnostics(data: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    rows = []
    for c in cols:
        s = data[c]
        s_nonnull = s.dropna()
        n = len(s)
        n_nonnull = len(s_nonnull)

        # If everything is missing, record and move on
        if n_nonnull == 0:
            rows.append({
                "metric": c,
                "n": n,
                "n_nonnull": 0,
                "pct_missing": 100.0,
                "pct_zero": np.nan,
                "min": np.nan,
                "p25": np.nan,
                "median": np.nan,
                "p75": np.nan,
                "max": np.nan,
                "skewness": np.nan,
            })
            continue

        pct_missing = 100.0 * (1 - (n_nonnull / n))
        pct_zero = 100.0 * (s_nonnull.eq(0).mean())

        # Use pandas skew (Fisher-Pearson)
        skewness = float(s_nonnull.skew())

        rows.append({
            "metric": c,
            "n": n,
            "n_nonnull": n_nonnull,
            "pct_missing": round(pct_missing, 2),
            "pct_zero": round(pct_zero, 2),
            "min": float(s_nonnull.min()),
            "p25": float(s_nonnull.quantile(0.25)),
            "median": float(s_nonnull.median()),
            "p75": float(s_nonnull.quantile(0.75)),
            "max": float(s_nonnull.max()),
            "skewness": round(skewness, 4),
        })

    return pd.DataFrame(rows)

diag_raw = compute_distribution_diagnostics(df, metric_cols)
save_df(diag_raw.sort_values("skewness", ascending=False), "distribution_diagnostics_raw")

print("Saved raw distribution diagnostics.")
print("\nMost skewed metrics (top 10):")
print(diag_raw.sort_values("skewness", ascending=False).head(10)[["metric", "skewness", "pct_zero", "median", "max"]])

Saved raw distribution diagnostics.

Most skewed metrics (top 10):
               metric  skewness  pct_zero        median            max
10            D_est_x   18.2728      0.00  1.458014e+01  285807.923987
14            D_est_y   18.2728      0.00  1.458014e+01  285807.923987
18              D_est   18.2728      0.00  1.458014e+01  285807.923987
1   total_path_length    8.2591     44.12  2.914317e+00    5366.956233
15            P_est_y    7.0455      0.00  1.705679e-07    5242.156716
11            P_est_x    7.0455      0.00  1.705679e-07    5242.156716
19              P_est    7.0455      0.00  1.705679e-07    5242.156716
0          num_frames    5.7225      0.00  2.000000e+00     210.000000
42  speed_trend_slope    3.7814      0.00  7.243178e-01      36.241242
17                  K    3.1928      0.00  1.340062e+02    3530.995913


In [4]:
# Block 4: Transformations (create new columns, keep raw intact)

def choose_transform_for_metric(series: pd.Series) -> str:
    """
    Decide transformation based on support:
    - If all values are >= 0 (ignoring NaNs), use log1p.
    - Else, use 'none' (leave as-is) for this addendum.
    """
    s = series.dropna()
    if len(s) == 0:
        return "none"
    if (s >= 0).all():
        return "log1p"
    return "none"

transform_plan = []
df_t = df.copy()

for c in metric_cols:
    plan = choose_transform_for_metric(df[c])
    transform_plan.append({"metric": c, "transform": plan})

    if plan == "log1p":
        # log1p is safe at zero; keeps scale reasonable for heavy-tailed positive metrics
        df_t[f"{c}__log1p"] = np.log1p(df[c])
    else:
        # We do not auto-transform metrics that can be negative in this minimal addendum.
        pass

transform_plan_df = pd.DataFrame(transform_plan)
save_df(transform_plan_df, "transform_plan")

# Quick counts
n_log = int((transform_plan_df["transform"] == "log1p").sum())
print(f"Transform plan saved. log1p applied to {n_log} metrics.")

# Save transformed dataset
save_df(df_t[[*ID_COLS, *[c for c in df_t.columns if c in metric_cols or c.endswith('__log1p')]]],
        "per_track_features_with_transforms")
print("Saved per_track_features_with_transforms.csv")


Transform plan saved. log1p applied to 38 metrics.
Saved per_track_features_with_transforms.csv


In [5]:
# Block 5: Build clean modeling table (post-transform) + handle duplicates + define X/y/groups

import numpy as np
import pandas as pd
from pathlib import Path

USE_SAVED = True

TRANSFORMED_PATH = OUT_DIR / "per_track_features_with_transforms.csv"

if USE_SAVED:
    df_transformed = pd.read_csv(TRANSFORMED_PATH)

# --- Safety checks ---
required_cols = {"stack", "track_id", "Genotype"}
missing_req = required_cols - set(df_transformed.columns)
if missing_req:
    raise ValueError(f"Block 5 expected columns missing: {missing_req}")

# Standardize label -> binary (MUT=1, WT=0)
y = (df_transformed["Genotype"].astype(str).str.upper() == "MUT").astype(int).values
groups = df_transformed["stack"].astype(str).values  # group-aware CV to avoid stack leakage

# --- Step 1: choose numeric feature columns (exclude identifiers) ---
id_cols = {"stack", "track_id", "Genotype"}
numeric_cols = df_transformed.select_dtypes(include=[np.number]).columns.tolist()
feature_cols = [c for c in numeric_cols if c not in id_cols]

# --- Step 2: drop extremely-missing columns (keep threshold explicit) ---
# Rationale: many columns are >58% missing; beyond ~60–70% the model becomes imputation-driven.
missing_pct = df_transformed[feature_cols].isna().mean() * 100

DROP_MISSING_OVER = 70.0
kept_cols = missing_pct[missing_pct <= DROP_MISSING_OVER].index.tolist()
dropped_cols = missing_pct[missing_pct > DROP_MISSING_OVER].index.tolist()

print(f"Features before missingness filter: {len(feature_cols)}")
print(f"Keeping (<= {DROP_MISSING_OVER:.0f}% missing): {len(kept_cols)}")
print(f"Dropping (> {DROP_MISSING_OVER:.0f}% missing): {len(dropped_cols)}")

# --- Step 3: handle “_x/_y” duplicated columns from merges ---
# If both alpha_x and alpha_y (etc.) are present, often one is redundant.
# Strategy: if both exist, create a single column as:
#   - prefer non-null from _x, else _y
# and then drop the suffix columns.
def collapse_xy_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    cols = df.columns
    x_cols = [c for c in cols if c.endswith("_x")]
    for cx in x_cols:
        base = cx[:-2]  # remove "_x"
        cy = base + "_y"
        if cy in cols:
            # create collapsed column only if base not already present
            if base not in cols:
                df[base] = df[cx].combine_first(df[cy])
            else:
                # if base exists, fill base from x/y where base is null
                df[base] = df[base].combine_first(df[cx]).combine_first(df[cy])

            # drop the suffix columns
            df.drop(columns=[cx, cy], inplace=True)

    return df

df_model = df_transformed[["stack", "track_id", "Genotype"] + kept_cols].copy()
df_model = collapse_xy_columns(df_model)

# Recompute feature columns after collapsing
numeric_cols2 = df_model.select_dtypes(include=[np.number]).columns.tolist()
feature_cols2 = [c for c in numeric_cols2 if c not in id_cols]

# --- Step 4: quick sanity report ---
print("\nAfter collapsing _x/_y duplicates:")
print("Model table shape:", df_model.shape)
print("Final feature count:", len(feature_cols2))

# Save the modeling-ready table for downstream blocks
model_path = OUT_DIR / "model_table_ready.csv"
df_model.to_csv(model_path, index=False)
print(f"Saved: {model_path}")

Features before missingness filter: 95
Keeping (<= 70% missing): 95
Dropping (> 70% missing): 0

After collapsing _x/_y duplicates:
Model table shape: (4263, 90)
Final feature count: 87
Saved: \Users\asus\Downloads\MSc Final Project Stuff\outputs_addendum\model_table_ready.csv


In [6]:
# Block 6: Leakage-safe baseline modeling (GroupKFold vs StratifiedKFold) with robust metrics

import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, balanced_accuracy_score, roc_auc_score
from sklearn.model_selection import cross_validate

# Paths 
MODEL_PATH = OUT_DIR / "model_table_ready.csv"

df_model = pd.read_csv(MODEL_PATH)

# Identify columns
id_cols = {"stack", "track_id", "Genotype"}
feature_cols = [c for c in df_model.columns if c not in id_cols and np.issubdtype(df_model[c].dtype, np.number)]

X = df_model[feature_cols]
y = (df_model["Genotype"].astype(str).str.upper() == "MUT").astype(int).values
groups = df_model["stack"].astype(str).values

print("Loaded model_table_ready.csv")
print("X shape:", X.shape, "| #features:", len(feature_cols))
print("y counts:", pd.Series(y).value_counts().to_dict())
print("Unique stacks:", len(np.unique(groups)))

# Pipeline: impute -> scale -> logistic regression
pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000, solver="liblinear", class_weight="balanced", random_state=42))
])

# Scorers
scoring = {
    "acc": "accuracy",
    "bal_acc": make_scorer(balanced_accuracy_score),
    "roc_auc": "roc_auc"
}

# CV strategies
cv_strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_group = GroupKFold(n_splits=min(5, len(np.unique(groups))))

def summarize_cv(name, cv, use_groups=False):
    res = cross_validate(
        pipe,
        X, y,
        cv=cv.split(X, y, groups=groups) if use_groups else cv,
        scoring=scoring,
        n_jobs=-1,
        return_train_score=False
    )
    summary = {
        "cv": name,
        "acc_mean": float(np.mean(res["test_acc"])),
        "acc_std":  float(np.std(res["test_acc"])),
        "bal_acc_mean": float(np.mean(res["test_bal_acc"])),
        "bal_acc_std":  float(np.std(res["test_bal_acc"])),
        "roc_auc_mean": float(np.mean(res["test_roc_auc"])),
        "roc_auc_std":  float(np.std(res["test_roc_auc"])),
    }
    return summary, res

summaries = []
raw_results = {}

s1, r1 = summarize_cv("StratifiedKFold", cv_strat, use_groups=False)
summaries.append(s1); raw_results["StratifiedKFold"] = r1

s2, r2 = summarize_cv("GroupKFold_by_stack", cv_group, use_groups=True)
summaries.append(s2); raw_results["GroupKFold_by_stack"] = r2

summary_df = pd.DataFrame(summaries).round(4)
print("\nCV comparison summary:")
display(summary_df)

# Save outputs
summary_path = OUT_DIR / "cv_baseline_logreg_summary.csv"
summary_df.to_csv(summary_path, index=False)
print(f"Saved: {summary_path}")

# Also save fold-level results for transparency
fold_rows = []
for cv_name, res in raw_results.items():
    n = len(res["test_acc"])
    for i in range(n):
        fold_rows.append({
            "cv": cv_name,
            "fold": i,
            "acc": res["test_acc"][i],
            "bal_acc": res["test_bal_acc"][i],
            "roc_auc": res["test_roc_auc"][i],
        })

fold_df = pd.DataFrame(fold_rows).round(6)
fold_path = OUT_DIR / "cv_baseline_logreg_folds.csv"
fold_df.to_csv(fold_path, index=False)
print(f"Saved: {fold_path}")

Loaded model_table_ready.csv
X shape: (4263, 87) | #features: 87
y counts: {0: 2815, 1: 1448}
Unique stacks: 16

CV comparison summary:


,cv,acc_mean,acc_std,bal_acc_mean,bal_acc_std,roc_auc_mean,roc_auc_std
0,StratifiedKFold,0.6803,0.0158,0.7227,0.0135,0.7808,0.0118
1,GroupKFold_by_stack,0.2867,0.2726,0.2944,0.2885,NaN,NaN


Saved: \Users\asus\Downloads\MSc Final Project Stuff\outputs_addendum\cv_baseline_logreg_summary.csv
Saved: \Users\asus\Downloads\MSc Final Project Stuff\outputs_addendum\cv_baseline_logreg_folds.csv


In [7]:
# Block 7: Leakage-aware permutation importance (GroupKFold by stack)
import numpy as np
import pandas as pd

from sklearn.model_selection import GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.inspection import permutation_importance

# --- Inputs assumed from prior blocks ---
# model_df: DataFrame that includes ["stack","Genotype"] + feature columns (already transformed)
# feature_cols: list of final feature columns used for modelling
# OUT_DIR: Path to output folder

model_df = pd.read_csv(OUT_DIR / "model_table_ready.csv")

# Build X, y, groups
X = model_df[feature_cols].copy()
y = (model_df["Genotype"] == "MUT").astype(int).values
groups = model_df["stack"].values

print("Loaded model_table_ready.csv")
print("X shape:", X.shape, "| #features:", X.shape[1])
print("y counts:", dict(pd.Series(y).value_counts()))
print("Unique stacks:", pd.Series(groups).nunique())

# --- Model ---
# Reason: permutation importance is easiest to interpret with a stable baseline classifier.
clf = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler()),
    ("model", LogisticRegression(max_iter=2000, solver="lbfgs"))
])

# --- GroupKFold (stack-level separation) ---
n_splits = min(5, pd.Series(groups).nunique())
gkf = GroupKFold(n_splits=n_splits)

importance_rows = []
fold_scores = []

for fold_idx, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups=groups), start=1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # If a fold ends up single-class in train or test (possible in edge cases), skip safely.
    if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:
        fold_scores.append({"fold": fold_idx, "balanced_acc": np.nan, "skipped": True})
        continue

    # Fit + evaluate
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    bal_acc = balanced_accuracy_score(y_test, y_pred)
    fold_scores.append({"fold": fold_idx, "balanced_acc": bal_acc, "skipped": False})

    # Permutation importance on held-out fold
    perm = permutation_importance(
        clf,
        X_test, y_test,
        scoring="balanced_accuracy",
        n_repeats=20,
        random_state=42,
        n_jobs=-1
    )

    for feat, mean_imp, std_imp in zip(feature_cols, perm.importances_mean, perm.importances_std):
        importance_rows.append({
            "fold": fold_idx,
            "feature": feat,
            "delta_bal_acc_mean": float(mean_imp),
            "delta_bal_acc_std": float(std_imp),
            "n_test": int(len(test_idx)),
            "test_stacks": int(pd.Series(groups[test_idx]).nunique()),
        })

importance_df = pd.DataFrame(importance_rows)
fold_scores_df = pd.DataFrame(fold_scores)

# Save fold performance 
fold_scores_df.to_csv(OUT_DIR / "perm_importance_groupkfold_fold_scores.csv", index=False)

if importance_df.empty:
    raise ValueError(
        "Permutation importance still produced no rows. "
        "This usually means every GroupKFold split had a single-class train/test set. "
        "If so, dataset may be genotype-separated by stack with no mixing across stacks."
    )

# Aggregate across folds: stability summary
importance_summary = (
    importance_df
    .groupby("feature", as_index=False)
    .agg(
        mean_delta=("delta_bal_acc_mean", "mean"),
        std_delta=("delta_bal_acc_mean", "std"),
        n_folds=("fold", "nunique")
    )
    .sort_values("mean_delta", ascending=False)
)

importance_df.to_csv(OUT_DIR / "permutation_importance_groupkfold_raw.csv", index=False)
importance_summary.to_csv(OUT_DIR / "permutation_importance_groupkfold_summary.csv", index=False)

print("\nSaved:")
print("-", OUT_DIR / "perm_importance_groupkfold_fold_scores.csv")
print("-", OUT_DIR / "permutation_importance_groupkfold_raw.csv")
print("-", OUT_DIR / "permutation_importance_groupkfold_summary.csv")

print("\nTop 20 features by mean drop in balanced accuracy (higher = more important):")
display(importance_summary.head(20))


Loaded model_table_ready.csv
X shape: (4263, 87) | #features: 87
y counts: {0: np.int64(2815), 1: np.int64(1448)}
Unique stacks: 16

Saved:
- \Users\asus\Downloads\MSc Final Project Stuff\outputs_addendum\perm_importance_groupkfold_fold_scores.csv
- \Users\asus\Downloads\MSc Final Project Stuff\outputs_addendum\permutation_importance_groupkfold_raw.csv
- \Users\asus\Downloads\MSc Final Project Stuff\outputs_addendum\permutation_importance_groupkfold_summary.csv

Top 20 features by mean drop in balanced accuracy (higher = more important):


,feature,mean_delta,std_delta,n_folds
0,D_est,0.000810,0.001001,3
8,P_est,0.000705,0.000990,3
15,area_delta_mean,0.000451,0.000925,3
32,early_speed_mean,0.000414,0.001049,3
73,r_peak__log1p,0.000000,0.000000,3
54,n_stationary__log1p,0.000000,0.000000,3
53,n_stationary,0.000000,0.000000,3
72,r_peak,0.000000,0.000000,3
84,temporal_persistence,-0.000068,0.000753,3
29,early_area_mean__log1p,-0.000535,0.000649,3


In [8]:
# Block 8: Method sanity summary
# Handles CV summaries where the CV strategy is stored as the index

import numpy as np
import pandas as pd
from pathlib import Path

MODEL_TABLE_PATH = OUT_DIR / "model_table_ready.csv"
CV_SUMMARY_PATH  = OUT_DIR / "cv_baseline_logreg_summary.csv"

# Load inputs
model_df = pd.read_csv(MODEL_TABLE_PATH)
cv_summary = pd.read_csv(CV_SUMMARY_PATH)

# Ensure identifier columns exist
id_cols = ["stack", "track_id", "Genotype"]
missing_ids = [c for c in id_cols if c not in model_df.columns]
if missing_ids:
    raise ValueError(f"model_table_ready.csv missing required columns: {missing_ids}")

# Identify numeric feature columns
feature_cols = [
    c for c in model_df.columns
    if c not in id_cols and pd.api.types.is_numeric_dtype(model_df[c])
]

# Basic dataset stats
n_rows     = len(model_df)
n_features = len(feature_cols)
n_stacks   = model_df["stack"].nunique()
geno_counts = model_df["Genotype"].value_counts(dropna=False).to_dict()

# Transform footprint
n_log1p = sum(c.endswith("_log1p") for c in feature_cols)

# Overall missingness
pct_missing_overall = float(model_df[feature_cols].isna().mean().mean() * 100)

# ---- CV SUMMARY ----
# If CV strategy is not a column, assume it's the index
if "cv_strategy" not in cv_summary.columns:
    cv_summary = cv_summary.reset_index().rename(columns={"index": "cv_strategy"})

required_cv_cols = {
    "cv_strategy",
    "acc_mean",
    "acc_std",
    "bal_acc_mean",
    "bal_acc_std"
}
missing_cv = required_cv_cols - set(cv_summary.columns)
if missing_cv:
    raise ValueError(f"CV summary missing required columns: {sorted(missing_cv)}")

# Build compact method summary
summary_rows = []

# Dataset summary
summary_rows.extend([
    {"section": "data", "metric": "n_rows", "value": n_rows},
    {"section": "data", "metric": "n_features", "value": n_features},
    {"section": "data", "metric": "n_stacks", "value": n_stacks},
    {"section": "data", "metric": "pct_missing_overall_features", "value": round(pct_missing_overall, 3)},
    {"section": "transforms", "metric": "n_log1p_features", "value": n_log1p},
])

# Label balance
for k, v in geno_counts.items():
    summary_rows.append({
        "section": "labels",
        "metric": f"count_{k}",
        "value": int(v)
    })

# CV performance summary
for _, r in cv_summary.iterrows():
    summary_rows.extend([
        {"section": "cv", "metric": f"{r['cv_strategy']}_acc_mean", "value": float(r["acc_mean"])},
        {"section": "cv", "metric": f"{r['cv_strategy']}_bal_acc_mean", "value": float(r["bal_acc_mean"])},
        {"section": "cv", "metric": f"{r['cv_strategy']}_acc_std", "value": float(r["acc_std"])},
        {"section": "cv", "metric": f"{r['cv_strategy']}_bal_acc_std", "value": float(r["bal_acc_std"])},
    ])

method_summary_df = pd.DataFrame(summary_rows)

# Save
out_path = OUT_DIR / "method_sanity_summary.csv"
method_summary_df.to_csv(out_path, index=False)

print("Saved:", out_path)
display(method_summary_df)


Saved: \Users\asus\Downloads\MSc Final Project Stuff\outputs_addendum\method_sanity_summary.csv


,section,metric,value
0,data,n_rows,4263.0000
1,data,n_features,87.0000
2,data,n_stacks,16.0000
3,data,pct_missing_overall_features,29.1190
4,transforms,n_log1p_features,38.0000
5,labels,count_WT,2815.0000
6,labels,count_MUT,1448.0000
7,cv,0_acc_mean,0.6803
8,cv,0_bal_acc_mean,0.7227
9,cv,0_acc_std,0.0158


In [9]:
# Block 9: Stability flags for permutation importance 
# - Uses the GroupKFold permutation importance summary.
# - Adds simple stability tags based on magnitude vs variability.
# - Exports "perm_importance_with_stability.csv"

import numpy as np
import pandas as pd

PERM_SUMMARY_PATH = OUT_DIR / "permutation_importance_groupkfold_summary.csv"

imp = pd.read_csv(PERM_SUMMARY_PATH)

# Validate expected columns
required_cols = {"feature", "mean_delta", "std_delta", "n_folds"}
missing = required_cols - set(imp.columns)
if missing:
    raise ValueError(f"Permutation importance summary missing columns: {sorted(missing)}")

# Ensure numeric types
for c in ["mean_delta", "std_delta", "n_folds"]:
    imp[c] = pd.to_numeric(imp[c], errors="coerce")

# Define a small epsilon to avoid division issues
eps = 1e-12

# Signal-to-noise style score (higher = more stable relative to noise)
imp["stability_score"] = imp["mean_delta"].abs() / (imp["std_delta"].abs() + eps)

# Simple tags 
# - "stable": consistently non-trivial and not wildly variable
# - "weak": tiny effect
# - "unstable": high variability relative to effect size
abs_mean = imp["mean_delta"].abs()

# Thresholds
weak_thr = abs_mean.quantile(0.50)  # median effect size
stable_thr = abs_mean.quantile(0.80)

imp["stability_tag"] = "unstable"
imp.loc[abs_mean < weak_thr, "stability_tag"] = "weak"
imp.loc[(abs_mean >= stable_thr) & (imp["stability_score"] >= 1.0), "stability_tag"] = "stable"

# Sort by importance magnitude
imp = imp.sort_values("mean_delta", ascending=False).reset_index(drop=True)

out_path = OUT_DIR / "perm_importance_with_stability.csv"
imp.to_csv(out_path, index=False)

print("Saved:", out_path)
print("\nTop 20 features (with stability tag):")
display(imp.head(20)[["feature", "mean_delta", "std_delta", "n_folds", "stability_score", "stability_tag"]])


Saved: \Users\asus\Downloads\MSc Final Project Stuff\outputs_addendum\perm_importance_with_stability.csv

Top 20 features (with stability tag):


,feature,mean_delta,std_delta,n_folds,stability_score,stability_tag
0,D_est,0.000810,0.001001,3,0.808956,weak
1,P_est,0.000705,0.000990,3,0.711502,weak
2,area_delta_mean,0.000451,0.000925,3,0.487622,weak
3,early_speed_mean,0.000414,0.001049,3,0.395047,weak
4,r_peak__log1p,0.000000,0.000000,3,0.000000,weak
5,n_stationary__log1p,0.000000,0.000000,3,0.000000,weak
6,n_stationary,0.000000,0.000000,3,0.000000,weak
7,r_peak,0.000000,0.000000,3,0.000000,weak
8,temporal_persistence,-0.000068,0.000753,3,0.089837,weak
9,early_area_mean__log1p,-0.000535,0.000649,3,0.823784,weak


In [10]:
# Block 10: Final summary 

from pathlib import Path
import pandas as pd
import numpy as np

PATH_METHOD = OUT_DIR / "method_sanity_summary.csv"
PATH_PERM   = OUT_DIR / "perm_importance_with_stability.csv"
PATH_CV     = OUT_DIR / "cv_baseline_logreg_summary.csv"

def _load_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Missing file: {path}")
    return pd.read_csv(path)

def normalize_cv_summary(cv_df: pd.DataFrame) -> pd.DataFrame:
    """
    Ensure we end up with a table that has:
      - cv_strategy (string)
      - acc_mean, acc_std, bal_acc_mean, bal_acc_std, roc_auc_mean, roc_auc_std (where available)
    Handles cases where:
      - cv_strategy was saved as index (e.g., first col is Unnamed: 0)
      - columns are prefixed with '0_' and '1_' 
      - roc_auc columns missing for GroupKFold (common)
    """
    df = cv_df.copy()

    # If cv_strategy is missing but there's an unnamed first column, treat it as cv_strategy.
    if "cv_strategy" not in df.columns:
        unnamed = [c for c in df.columns if str(c).startswith("Unnamed")]
        if unnamed:
            df = df.rename(columns={unnamed[0]: "cv_strategy"})
        else:
            # If still missing, but there is exactly 1 non-metric column, use it
            non_numeric_cols = [c for c in df.columns if not pd.api.types.is_numeric_dtype(df[c])]
            if len(non_numeric_cols) == 1:
                df = df.rename(columns={non_numeric_cols[0]: "cv_strategy"})

    # If still missing, create a placeholder based on row order (last resort)
    if "cv_strategy" not in df.columns:
        df["cv_strategy"] = [f"cv_{i}" for i in range(len(df))]

    # Case A: already in "tidy" format
    tidy_cols = {"acc_mean","acc_std","bal_acc_mean","bal_acc_std","roc_auc_mean","roc_auc_std"}
    if any(c in df.columns for c in tidy_cols):
        # Ensure all expected cols exist (fill with NaN if absent)
        for c in tidy_cols:
            if c not in df.columns:
                df[c] = np.nan
        return df[["cv_strategy","acc_mean","acc_std","bal_acc_mean","bal_acc_std","roc_auc_mean","roc_auc_std"]]

    # Case B:
    # We'll detect prefixes "<rowIndex>_<metric>" and rebuild a tidy table.
    split_cols = []
    for c in df.columns:
        if isinstance(c, str) and "_" in c and c.split("_", 1)[0].isdigit():
            split_cols.append(c)

    if split_cols:
        rows = []
        for _, row in df.iterrows():
            cv_name = str(row["cv_strategy"])
            # Gather all columns that begin with this row index if possible
            out = {"cv_strategy": cv_name}
            # Pull metrics by checking any column that endswith metric name
            for metric in ["acc_mean","acc_std","bal_acc_mean","bal_acc_std","roc_auc_mean","roc_auc_std"]:
                candidates = [c for c in df.columns if c.endswith(f"_{metric}")]
                if candidates:
                    # If multiple, pick the first non-null value in this row
                    val = np.nan
                    for cc in candidates:
                        if pd.notna(row[cc]):
                            val = row[cc]
                            break
                    out[metric] = val
                else:
                    out[metric] = np.nan
            rows.append(out)

        tidy = pd.DataFrame(rows)
        return tidy[["cv_strategy","acc_mean","acc_std","bal_acc_mean","bal_acc_std","roc_auc_mean","roc_auc_std"]]

    # Case C: unknown format — try to coerce numeric metric columns and keep them
    numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    keep = ["cv_strategy"] + numeric_cols
    return df[keep]

# ---- Load upstream outputs ----
method_df = _load_csv(PATH_METHOD)
perm_df   = _load_csv(PATH_PERM)
cv_raw    = _load_csv(PATH_CV)
cv_tidy   = normalize_cv_summary(cv_raw)

# ---- Build a compact "report-ready" checklist table ----
items = []

# data stats
def _pick(section, metric):
    m = method_df[(method_df["section"] == section) & (method_df["metric"] == metric)]
    return float(m["value"].iloc[0]) if len(m) else np.nan

items.append(("Data", "Tracks (rows)", _pick("data","n_rows")))
items.append(("Data", "Features used", _pick("data","n_features")))
items.append(("Data", "Stacks (videos)", _pick("data","n_stacks")))
items.append(("Preprocess", "Overall missingness (%)", _pick("data","pct_missing_overall_features")))
items.append(("Preprocess", "log1p-transformed features", _pick("transforms","n_log1p_features")))
items.append(("Labels", "WT count", _pick("labels","count_WT")))
items.append(("Labels", "MUT count", _pick("labels","count_MUT")))

# CV best strategy (by balanced accuracy if available)
best_cv_strategy = None
best_bal = np.nan
if "bal_acc_mean" in cv_tidy.columns:
    tmp = cv_tidy.dropna(subset=["bal_acc_mean"]).sort_values("bal_acc_mean", ascending=False)
    if len(tmp):
        best_cv_strategy = str(tmp["cv_strategy"].iloc[0])
        best_bal = float(tmp["bal_acc_mean"].iloc[0])

items.append(("CV", "Best CV (by bal_acc_mean)", best_cv_strategy if best_cv_strategy is not None else "NA"))
items.append(("CV", "Best bal_acc_mean", best_bal))

# Permutation importance headline
top_perm = perm_df.sort_values("mean_delta", ascending=False).head(10)
top_feat = str(top_perm["feature"].iloc[0]) if len(top_perm) else "NA"
items.append(("Importance", "Top feature (perm, bal_acc drop)", top_feat))

summary_df = pd.DataFrame(items, columns=["section","item","value"])
out_path = OUT_DIR / "final_addendum_summary.csv"
summary_df.to_csv(out_path, index=False)

print(f"Saved: {out_path}")
print("\nCV (tidy) preview:")
print(cv_tidy)

print("\nFinal addendum summary preview:")
print(summary_df)


Saved: \Users\asus\Downloads\MSc Final Project Stuff\outputs_addendum\final_addendum_summary.csv

CV (tidy) preview:
           cv_strategy  acc_mean  acc_std  bal_acc_mean  bal_acc_std  \
0      StratifiedKFold    0.6803   0.0158        0.7227       0.0135   
1  GroupKFold_by_stack    0.2867   0.2726        0.2944       0.2885   

   roc_auc_mean  roc_auc_std  
0        0.7808       0.0118  
1           NaN          NaN  

Final addendum summary preview:
      section                              item            value
0        Data                     Tracks (rows)           4263.0
1        Data                     Features used             87.0
2        Data                   Stacks (videos)             16.0
3  Preprocess           Overall missingness (%)           29.119
4  Preprocess        log1p-transformed features             38.0
5      Labels                          WT count           2815.0
6      Labels                         MUT count           1448.0
7          CV       

In [11]:
# Block 11: Method justification checklist

import pandas as pd

justification_rows = [

    # Data characteristics
    {
        "decision": "Log1p transformation of skewed features",
        "why_chosen": (
            "Many per-track metrics showed strong positive skew and heavy tails. "
            "Log1p stabilizes variance while preserving zero values, improving "
            "comparability and interpretability."
        ),
        "alternatives_considered": "No transform; Box–Cox (requires strictly positive data)",
        "assumptions": "Monotonic transform preserves rank ordering",
        "limitations": "Does not guarantee normality; interpretation shifts to log scale"
    },

    # Statistical testing
    {
        "decision": "Use of nonparametric tests (e.g. Mann–Whitney U) for group comparisons",
        "why_chosen": (
            "Preliminary inspection showed non-Gaussian distributions and unequal variances. "
            "Rank-based tests avoid parametric assumptions under these conditions."
        ),
        "alternatives_considered": "Two-sample t-test; Welch’s t-test",
        "assumptions": "Independence; ordinal comparability",
        "limitations": "Sensitive only to monotone differences; reduced power under normality"
    },

    # Model evaluation
    {
        "decision": "Group-aware cross-validation by video stack",
        "why_chosen": (
            "Tracks from the same video are not statistically independent. "
            "GroupKFold prevents information leakage across folds."
        ),
        "alternatives_considered": "Standard StratifiedKFold",
        "assumptions": "Stack-level grouping captures dominant dependence structure",
        "limitations": "Higher variance; fewer effective folds"
    },

    # Feature importance
    {
        "decision": "Permutation importance under group-aware CV",
        "why_chosen": (
            "Permutation importance evaluates feature contribution without assuming "
            "linearity and reflects performance sensitivity under realistic validation."
        ),
        "alternatives_considered": "Model coefficients; impurity-based importance",
        "assumptions": "Feature perturbation reflects information content",
        "limitations": "Unstable with correlated features; fold-dependent variability"
    },

    # Modeling choice
    {
        "decision": "Logistic regression as baseline classifier",
        "why_chosen": (
            "Provides a transparent linear baseline with interpretable coefficients "
            "and stable optimization under high-dimensional features."
        ),
        "alternatives_considered": "Tree ensembles; kernel classifiers",
        "assumptions": "Approximate linear separability in feature space",
        "limitations": "Limited capacity for complex nonlinear interactions"
    }
]

justification_df = pd.DataFrame(justification_rows)

out_path = OUT_DIR / "method_justification_checklist.csv"
justification_df.to_csv(out_path, index=False)

print(f"Saved method justification checklist to:\n{out_path}")
display(justification_df)

Saved method justification checklist to:
\Users\asus\Downloads\MSc Final Project Stuff\outputs_addendum\method_justification_checklist.csv


,decision,why_chosen,alternatives_considered,assumptions,limitations
0,Log1p transformation of skewed features,Many per-track metrics showed strong positive ...,No transform; Box–Cox (requires strictly posit...,Monotonic transform preserves rank ordering,Does not guarantee normality; interpretation s...
1,Use of nonparametric tests (e.g. Mann–Whitney ...,Preliminary inspection showed non-Gaussian dis...,Two-sample t-test; Welch’s t-test,Independence; ordinal comparability,Sensitive only to monotone differences; reduce...
2,Group-aware cross-validation by video stack,Tracks from the same video are not statistical...,Standard StratifiedKFold,Stack-level grouping captures dominant depende...,Higher variance; fewer effective folds
3,Permutation importance under group-aware CV,Permutation importance evaluates feature contr...,Model coefficients; impurity-based importance,Feature perturbation reflects information content,Unstable with correlated features; fold-depend...
4,Logistic regression as baseline classifier,Provides a transparent linear baseline with in...,Tree ensembles; kernel classifiers,Approximate linear separability in feature space,Limited capacity for complex nonlinear interac...
